# Listeners must have arguments "event" and "metadata" with proper types

In [1]:
from morpheus.project.domain.events.ProjectEvents.ProjectEvents import ProjectCreatedEvent
from morpheus.common.types.event_sourcing.EventMetadata import EventMetadata
from morpheus.common.infrastructure.event_sourcing.EventPublisher import EventPublisher, listen_to


class Invalid:
  pass


class ProjectorInvalid0:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self):
    print('Created project')


class ProjectorInvalid1:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, invalid: ProjectCreatedEvent, metadata: EventMetadata):
    print('Created project')


class ProjectorInvalid2:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, event: ProjectCreatedEvent, invalid: EventMetadata):
    print('Created project')


class ProjectorInvalid3:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, event: Invalid, metadata: EventMetadata):
    print('Created project')


class ProjectorInvalid4:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, event: ProjectCreatedEvent, metadata: Invalid):
    print('Created project')


event_publisher = EventPublisher()
try:
  projector = ProjectorInvalid0()
  event_publisher.register(projector)
except Exception as e:
  print(e)

try:
  projector = ProjectorInvalid1()
  event_publisher.register(projector)
except Exception as e:
  print(e)

try:
  projector = ProjectorInvalid2()
  event_publisher.register(projector)
except Exception as e:
  print(e)

try:
  projector = ProjectorInvalid3()
  event_publisher.register(projector)
except Exception as e:
  print(e)

try:
  projector = ProjectorInvalid4()
  event_publisher.register(projector)
except Exception as e:
  print(e)

Event listener <bound method ProjectorInvalid0.on_project_created of <__main__.ProjectorInvalid0 object at 0x14a40e570>> must have an argument called "event" that should by type hinted with a subclass of EventBase
Event listener <bound method ProjectorInvalid1.on_project_created of <__main__.ProjectorInvalid1 object at 0x14a71e600>> must have an argument called "event" that should by type hinted with a subclass of EventBase
Event listener <bound method ProjectorInvalid2.on_project_created of <__main__.ProjectorInvalid2 object at 0x14a40e570>> must have an argument called "metadata" that should by type hinted with a subclass of EventMetadata
Event listener <bound method ProjectorInvalid3.on_project_created of <__main__.ProjectorInvalid3 object at 0x14a740b90>> must have an argument called "event" that should by type hinted with a subclass of EventBase
Event listener <bound method ProjectorInvalid4.on_project_created of <__main__.ProjectorInvalid4 object at 0x14a40e570>> must have an arg

# Cannot register same listener twice

In [2]:
class Projector:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, event: ProjectCreatedEvent, metadata: EventMetadata):
    print(f'Created project {event.get_project_id()} with metadata {metadata}')


try:
  event_publisher = EventPublisher()
  projector = Projector()
  event_publisher.register(projector)
  event_publisher.register(projector)
except Exception as e:
  print(e)

Event handler already registered


# Create projects

In [3]:
from morpheus.common.types.identity.Identity import UserId
from morpheus.project.infrastructure.persistence.ProjectSummaryRepository import project_summary_repository
from morpheus.project.infrastructure.event_sourcing.ProjectEventStore import project_event_store
from morpheus.project.types.Project import ProjectId, Name, Description, Tags
from morpheus.project.application.write.Project import CreateProjectCommand, CreateProjectCommandHandler

for i in range(10):
  create_project_command = CreateProjectCommand(
    project_id=ProjectId.new(),
    name=Name.from_str(f'Project {i}'),
    description=Description.from_str(f'Description {i}'),
    tags=Tags.from_list([f'tag{i}', f'tag{i + 1}']),
    user_id=UserId.new()
  )
  CreateProjectCommandHandler.handle(create_project_command)

event_envelopes = project_event_store.get_all_events_ordered_by_version()
print(f'Found {len(event_envelopes)} events')
for event_envelope in event_envelopes:
  print(f'"{event_envelope.get_event().get_event_name().to_str()}" occurred at {event_envelope.event.get_occurred_at().to_iso_with_timezone()}')

project_summaries = project_summary_repository.find_all()
print(f'Found {len(project_summaries)} project summaries')
for project_summary in project_summaries:
  print(f'Project {project_summary.project_id.to_str()}: {project_summary.project_name.to_str()}')

Found 700 events
"Project Created" occurred at 2024-07-08T10:10:10.094033+02:00
"Project Created" occurred at 2024-07-10T10:26:32.268323+02:00
"Project Created" occurred at 2024-07-15T13:14:17.536861+02:00
"Project Created" occurred at 2024-08-19T11:40:07.566448+02:00
"Project Created" occurred at 2024-08-19T11:40:07.587290+02:00
"Project Created" occurred at 2024-08-19T11:40:07.589498+02:00
"Project Created" occurred at 2024-08-19T11:40:07.591553+02:00
"Project Created" occurred at 2024-08-19T11:40:07.593618+02:00
"Project Created" occurred at 2024-08-19T11:40:07.595619+02:00
"Project Created" occurred at 2024-08-19T11:40:07.597472+02:00
"Project Created" occurred at 2024-08-19T11:40:07.599804+02:00
"Project Created" occurred at 2024-08-19T11:40:07.602123+02:00
"Project Created" occurred at 2024-08-19T11:40:07.604680+02:00
"Project Created" occurred at 2024-08-19T11:40:25.610977+02:00
"Project Created" occurred at 2024-08-19T11:42:14.559930+02:00
"Project Created" occurred at 2024-08-

# Reproject project summaries

In [4]:
from morpheus.project.presentation.cli.ProjectionCliCommands import ReprojectProjectSummariesCliCommand

ReprojectProjectSummariesCliCommand.run()

Reprojecting project summaries


------------------------------


Found 700 events


Reset projection


Successfully reprojected project summaries


In [5]:
project_summaries = project_summary_repository.find_all()
print(f'Found {len(project_summaries)} project summaries')
for project_summary in project_summaries:
  print(f'Project {project_summary.project_id.to_str()}: {project_summary.project_name.to_str()}')

Found 41 project summaries
Project 8cb9fced-e3d1-42f7-bcda-0e1aa4e4c70e: Rio Primero
Project 2a8caac3-1c25-4133-a7a5-91a9a1922acd: Project 0
Project 2aff8082-06bd-4f2b-a73a-e79dec3de6ef: Project 1
Project bb828686-eca0-49ab-aa16-e10141b80265: Project 2
Project 3fdc6d0b-eaff-4f47-9661-502d16b0bd69: Project 3
Project d4969478-8794-4f8b-94a2-a7665dbb2de1: Project 4
Project 802728d2-26c6-493e-a676-5da1c475a6b7: Project 5
Project a301367f-aca5-4bb3-9517-d873082279d9: Project 6
Project bd8e9ef9-443f-4281-8e36-79ee31317c2d: Project 7
Project 2795b484-e8fd-44a3-ab0e-4aa27dc7f803: Project 8
Project 9c3ec8ca-8871-499e-8403-b430784ca0dd: Project 9
Project 28a73512-389c-44ca-9b56-f0350a77e273: Project 1
Project aa1645b6-780a-4948-bd18-87d1343f4f7a: Project 1
Project 4c190e2a-effb-4103-859c-cc1a964e9dc4: Project 1
Project 9334b31f-3c71-4dfe-b3d3-2600ff110f3b: Rio Primero 123
Project d91e2e77-a95f-4192-9d89-dcec7642f9c4: Rio Primero 123
Project 891307a5-0027-4f03-8190-8b994eba1cdb: Project 0
Project